**1.Importa las librerías requeridas.**

In [29]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

**2.Lee el archivo CSV llamado empleadosRETO.csv y coloca los datos en un frame de Pandas llamado EmpleadosAttrition.**



In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [3]:
EmpleadosAttrition=pd.read_csv('/content/gdrive/MyDrive/empleadosRETO.csv')

**3.	Elimina las columnas que, con alta probabilidad, no tienen relación alguna con la salida. Hay algunas columnas que contienen información que no ayuda a definir el desgaste de un empleado**

In [4]:
EmpleadosAttrition = EmpleadosAttrition.drop(columns=['EmployeeCount', 'EmployeeNumber','Over18','StandardHours'], axis=1)

**4.	Analizando la información proporcionada, detectaste que no se cuenta con los años que el empelado lleva en la compañía y parece ser un buen dato. Dicha cantidad se puede calcular con la fecha de contratación ‘HiringDate’:**

*a.	Crea una columna llamada Year y obtén el año de contratación del empleado a partir de su fecha ‘HiringDate’. No se te olvide que debe ser un entero*

In [5]:
EmpleadosAttrition['Year'] = EmpleadosAttrition['HiringDate'].str[-4:].astype(int)

*b.	Crea una columna llamada YearsAtCompany que contenga los años que el empleado lleva en la compañía hasta el año 2018. Para su cálculo, usa la variable Year que acabas de crear.*

In [6]:
EmpleadosAttrition['YearsAtCompany']=2018-EmpleadosAttrition['Year']

**5.	La DistanceFromHome está dada en kilómetros, pero tiene las letras “km” al final y así no puede ser entera:**

*a.	Renombra la variable DistanceFromHome a DistanceFromHome_km.*


In [7]:
EmpleadosAttrition.rename(columns = {'DistanceFromHome': 'DistanceFromHome_km'}, inplace = True)

*b.	Crea una nueva variable DistanceFromHome que sea entera, es decir, solo con números.*

In [8]:
EmpleadosAttrition['DistanceFromHome'] =EmpleadosAttrition['DistanceFromHome_km'].str.split(pat=' ').str[0].astype(int)

**6.	Borra las columnas Year, HiringDate y DistanceFromHome_km debido a que ya no son útiles.**

In [9]:
EmpleadosAttrition = EmpleadosAttrition.drop(columns=['Year', 'HiringDate','DistanceFromHome_km'], axis=1)

**7.	Aprovechando los ajustes que se están haciendo, la empresa desea saber si todos los departamentos tienen un ingreso promedio similar. Genera una nuevo frame llamado SueldoPromedioDepto que contenga el MonthlyIncome promedio por departamento de los empleados y colócalo en una variable llamada SueldoPromedio. Esta tabla solo es informativa, no la vas a utilizar en el set de datos que estás construyendo.**

In [10]:
#continents_group = continents_2019.groupby(['Continent']).mean()
SueldoPromedioDepto=EmpleadosAttrition[['Department','MonthlyIncome']]
SueldoPromedioDepto=SueldoPromedioDepto.groupby(['Department']).mean('MontlyIncome')

In [11]:
SueldoPromedioDepto.rename(columns={'MonthlyIncome':'SueldoPromedio'}, inplace = True)

In [12]:
SueldoPromedioDepto

,SueldoPromedio
Department,
Human Resources,6239.888889
Research & Development,6804.149813
Sales,7188.250000


**8.	La variable MonthlyIncome tiene un valor numérico muy grande comparada con las otras variables. Escala dicha variable para que tenga un valor entre 0 y 1.**

In [13]:
EmpleadosAttrition['MonthlyIncome']=(EmpleadosAttrition['MonthlyIncome']-EmpleadosAttrition['MonthlyIncome'].min())/(EmpleadosAttrition['MonthlyIncome'].max()-EmpleadosAttrition['MonthlyIncome'].min())

**9.	Todo parece indicar que las variables categóricas que quedan sí son importantes para obtener la variable de salida. Convierte todas las variables categóricas que quedan a numéricas:**

In [14]:
BusinessTravel_OH = pd.get_dummies(EmpleadosAttrition.BusinessTravel, prefix = 'classTravel')
Department_OH = pd.get_dummies(EmpleadosAttrition.Department, prefix = 'classDepartment')
EducationField_OH = pd.get_dummies(EmpleadosAttrition.EducationField, prefix = 'classEducation')
Gender_OH = pd.get_dummies(EmpleadosAttrition.Gender, prefix = 'classGender')
JobRole_OH = pd.get_dummies(EmpleadosAttrition.JobRole, prefix = 'classRole')
MaritalStatus_OH = pd.get_dummies(EmpleadosAttrition.MaritalStatus, prefix = 'classMaritalStatus')
Attrition_OH = pd.get_dummies(EmpleadosAttrition.Attrition, prefix = 'classAttrition')
EmpleadosAttrition = pd.concat([EmpleadosAttrition, BusinessTravel_OH,Department_OH,Gender_OH,JobRole_OH,MaritalStatus_OH,Attrition_OH], axis=1)

In [15]:
EmpleadosAttrition = EmpleadosAttrition.drop(columns=['Department','BusinessTravel','EducationField','Gender','JobRole','MaritalStatus','Attrition'], axis=1)

**10.	Ahora debes hacer la evaluación de las variables para quedarte con las mejores. Calcula la correlación lineal de cada una de las variables con respecto al Attrition.**

In [21]:
correlations = EmpleadosAttrition.corr()['classAttrition_Yes'].abs()

<ipython-input-21-65659f021c34>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlations = EmpleadosAttrition.corr()['classAttrition_Yes'].abs()


In [22]:
correlations

Age                                       0.212121
Education                                 0.055531
EnvironmentSatisfaction                   0.124327
JobInvolvement                            0.166785
JobLevel                                  0.214266
JobSatisfaction                           0.164957
MonthlyIncome                             0.194936
NumCompaniesWorked                        0.009082
PercentSalaryHike                         0.060880
PerformanceRating                         0.006471
RelationshipSatisfaction                  0.030945
TotalWorkingYears                         0.213329
TrainingTimesLastYear                     0.070884
WorkLifeBalance                           0.021723
YearsInCurrentRole                        0.203918
YearsSinceLastPromotion                   0.069000
YearsAtCompany                            0.176001
DistanceFromHome                          0.052732
classTravel_Non-Travel                    0.100698
classTravel_Travel_Frequently  

**11.	Selecciona solo aquellas variables que tengan una correlación mayor o igual a 0.1, dejándolas en otro frame llamado EmpleadosAttritionFinal. No olvides mantener la variable de salida Attrition; esto es equivalente a borrar las que no cumplen con el límite.**

In [25]:
variables_seleccion= correlations[correlations >= 0.1].index.tolist()
variables_seleccion

['Age',
 'EnvironmentSatisfaction',
 'JobInvolvement',
 'JobLevel',
 'JobSatisfaction',
 'MonthlyIncome',
 'TotalWorkingYears',
 'YearsInCurrentRole',
 'YearsAtCompany',
 'classTravel_Non-Travel',
 'classRole_Healthcare Representative',
 'classRole_Laboratory Technician',
 'classRole_Research Director',
 'classRole_Sales Representative',
 'classMaritalStatus_Divorced',
 'classMaritalStatus_Single',
 'classAttrition_No',
 'classAttrition_Yes']

In [ ]:
EmpleadosAttritionFinal = EmpleadosAttrition[EmpleadosAttrition.columns[EmpleadosAttrition.columns.isin(variables_seleccion)]]
EmpleadosAttritionFinal

**12.	Crea una nueva variable llamada EmpleadosAttritionPCA formada por los componentes principales del frame EmpleadosAttritionFinal. Recuerda que el resultado del proceso PCA es un numpy array, por lo que, para hacer referencia a una columna, por ejemplo, la 0, puedes usar la instrucción EmpleadosAttritionPCA[:,0]).**

In [32]:
pca=PCA()
features = EmpleadosAttritionFinal.drop(columns=['classAttrition_No','classAttrition_Yes'])
std_features = (features - features.mean()) / features.std()
EmpleadosAttritionPCA_array = pca.fit_transform(std_features)
EmpleadosAttritionPCA = pd.DataFrame(data=EmpleadosAttritionPCA_array, columns=[f'PC{i}' for i in range(1, pca.n_components_ + 1)])

In [34]:
EmpleadosAttritionPCA_array

array([[ 3.99311116, -0.15734526, -1.24397533, ...,  0.81768923,
        -0.84012585, -0.19958489],
       [-0.9417824 , -0.82523951, -0.82352495, ..., -0.19988228,
        -0.03444212,  0.16921092],
       [-3.46471934,  2.82104064,  1.54002145, ...,  0.03180078,
         0.02751771, -0.04919428],
       ...,
       [ 2.15223018, -0.85870473,  0.61770239, ..., -0.33467457,
         0.1014789 , -0.07874062],
       [-2.27559447, -0.6790646 , -1.05890702, ..., -0.02919313,
        -0.05737142, -0.14456844],
       [-0.96715142, -0.4679379 ,  0.2615762 , ...,  0.16179287,
        -0.19473826, -0.13169851]])

In [38]:
EmpleadosAttritionPCA

,C0,C1,C2,C3,C4,C5,C6,C7,C8
0,3.993111,-0.157345,-1.243975,-0.570848,1.802635,0.950790,-1.991383,-1.624437,-0.324018
1,-0.941782,-0.825240,-0.823525,-0.234262,0.325704,1.615159,-0.006279,-0.070274,-0.097343
2,-3.464719,2.821041,1.540021,-0.211892,-0.904759,1.977015,-0.206500,-0.103064,0.471887
3,1.649728,1.520148,-1.766818,1.351874,-1.284307,-0.817613,1.188014,1.070444,0.783582
4,1.107130,0.231638,0.096339,-0.120364,-0.317108,0.325435,0.338598,-0.672703,-0.064842
...,...,...,...,...,...,...,...,...,...
395,-1.821931,-1.250477,0.045552,-0.770872,0.045771,-1.262513,0.550930,-0.976972,-0.439782
396,-1.354000,0.214580,0.399823,1.206966,0.647436,0.395685,-0.116952,0.479058,-1.736018
397,2.152230,-0.858705,0.617702,-0.440001,0.660981,1.851690,-0.339227,-1.807798,-0.294863
398,-2.275594,-0.679065,-1.058907,-1.206304,0.865396,-1.321213,-0.253028,-0.352405,-0.232747


**13.	Agrega el mínimo número de Componentes Principales en columnas del frame EmpleadosAttritionPCA que logren explicar el 80% de la varianza, al frame EmpleadosAttritionFinal. Puedes usar la instrucción assign, columna por columna, llamando a cada una C0, C1, etc., hasta las que vayas a agregar.**

In [37]:
cumulative_var_ratio = np.cumsum(pca.explained_variance_ratio_)
num_components = np.argmax(cumulative_var_ratio >= 0.8) + 1
EmpleadosAttritionPCA = pd.DataFrame(data=EmpleadosAttritionPCA_array[:, :num_components],
                                      columns=[f'C{i}' for i in range(num_components)])
EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(**EmpleadosAttritionPCA)
EmpleadosAttritionFinal

,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,TotalWorkingYears,YearsInCurrentRole,YearsAtCompany,classTravel_Non-Travel,...,classAttrition_Yes,C0,C1,C2,C3,C4,C5,C6,C7,C8
0,50,4,3,4,4,0.864269,32,4,5,0,...,0,3.993111,-0.157345,-1.243975,-0.570848,1.802635,0.950790,-1.991383,-1.624437,-0.324018
1,36,2,3,2,2,0.207340,7,2,3,0,...,0,-0.941782,-0.825240,-0.823525,-0.234262,0.325704,1.615159,-0.006279,-0.070274,-0.097343
2,21,2,3,1,2,0.088062,1,0,1,0,...,1,-3.464719,2.821041,1.540021,-0.211892,-0.904759,1.977015,-0.206500,-0.103064,0.471887
3,52,2,3,3,2,0.497574,18,6,8,0,...,0,1.649728,1.520148,-1.766818,1.351874,-1.284307,-0.817613,1.188014,1.070444,0.783582
4,33,2,3,3,3,0.664470,15,6,7,0,...,1,1.107130,0.231638,0.096339,-0.120364,-0.317108,0.325435,0.338598,-0.672703,-0.064842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,33,3,3,1,4,0.075248,8,4,5,0,...,1,-1.821931,-1.250477,0.045552,-0.770872,0.045771,-1.262513,0.550930,-0.976972,-0.439782
396,31,2,1,2,3,0.187197,4,2,2,0,...,1,-1.354000,0.214580,0.399823,1.206966,0.647436,0.395685,-0.116952,0.479058,-1.736018
397,37,2,3,3,4,0.589327,10,8,10,0,...,0,2.152230,-0.858705,0.617702,-0.440001,0.660981,1.851690,-0.339227,-1.807798,-0.294863
398,38,4,3,1,3,0.121124,7,0,0,0,...,0,-2.275594,-0.679065,-1.058907,-1.206304,0.865396,-1.321213,-0.253028,-0.352405,-0.232747


In [42]:
EmpleadosAttritionFinal.to_csv('/content/gdrive/MyDrive/EmpleadosAttritionFinal.csv', index=False)